In [1]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

model_id = "IDEA-Research/grounding-dino-base"

processor = AutoProcessor.from_pretrained(model_id)
device = "cuda"
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)


2024-08-27 13:28:45.619858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 13:28:45.685166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-27 13:28:45.685606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-27 13:28:45.870843: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-27 13:28:46.824379: W tensorflow/compiler/tf

In [2]:
from pathlib import Path

file_list = sorted(Path("plot/f1").glob("*jpg"))
image_url = str(file_list[3])
image = Image.open(image_url)

# Check for cats and remote controls
text = "a pen."

inputs = processor(images=image, text=text, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
target_sizes = torch.tensor([image.size[::-1]])
results = processor.image_processor.post_process_object_detection(
    outputs, threshold=0.35, target_sizes=target_sizes
)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 1) for i in box.tolist()]
    conf = round(score.item(), 2)

    print(f"Detected {label.item()} with confidence " f"{round(score.item(), 2)} at location {box}")

Detected 1 with confidence 0.42 at location [538.0, 359.1, 682.0, 407.5]


In [5]:
import ultralytics
from ultralytics.utils.plotting import Annotator

annotator = Annotator(image)
annotator.box_label(box, label=str(conf), color=(0, 204, 0)) # The bounding box coordinates (x1, y1, x2, y2)

import cv2
result = annotator.result()
cv2.imwrite("data/plot.jpg", cv2.cvtColor(result, cv2.COLOR_RGB2BGR), [cv2.IMWRITE_JPEG_QUALITY, 100])

True